In [96]:
import numpy as np
import pandas as pd

def solve_sabr_pde_explicit(
    f_max=200.0,      # Maximum f value
    alpha_max=1.0,  # Maximum alpha value
    T=1.0,          # Maturity
    beta=1,       # SABR beta parameter
    rho=0.07,       # SABR correlation
    nu=0.5,         # SABR vol-of-vol
    r=0.0,          # Risk-free rate (assume 0 for simplicity)
    N_f=50,         # Number of spatial steps in f
    N_alpha=50,     # Number of spatial steps in alpha
    N_t=2500,         # Number of time steps
    strike=100.0      # Strike for a European call payoff
):
    """
    Solve the SABR PDE for a European call option using an explicit finite difference scheme.

    Returns: 
        f_values, alpha_values, V array of shape (N_t+1, N_alpha+1, N_f+1)
        where V[n, j, i] corresponds to V at time index n, alpha index j, f index i.
    """
    # 1. Define grids
    df = f_max / N_f
    dalpha = alpha_max / N_alpha
    dt = T / N_t

    # f-values and alpha-values for the grid
    f_values = np.linspace(0, f_max, N_f+1)
    alpha_values = np.linspace(0, alpha_max, N_alpha+1)
    time_values = np.linspace(0, T, N_t+1)  # We'll go backwards

    # 2. Initialize the solution array: 
    #    V[n, j, i] ~ V at time index n, alpha index j, f index i
    V = np.zeros((N_t+1, N_alpha+1, N_f+1))

    # 3. Terminal (payoff) condition at t = T
    #    For a European call option, payoff = max(f - K, 0).
    #    Here we do not explicitly use alpha in the payoff.
    for i, f_i in enumerate(f_values):
        payoff = max(f_i - strike, 0.0)
        V[-1, :, i] = payoff  # At t = T, for all alpha

    # 4. Boundary conditions:
    #    (A) f = 0 => payoff is 0 for a call
    #    (B) f = f_max => often approximate, e.g. at large f, call ~ f. 
    #    (C) alpha = 0 => effectively zero vol, solution evolves as in a Black-Scholes degenerate form
    #    (D) alpha = alpha_max => approximate boundary, often higher than typical alpha
    #
    # For simplicity, we'll set boundary values to remain equal to payoff or a certain condition.

    # We'll step in time from n = N_t-1 down to 0
    for n in reversed(range(N_t)):  # n: time index
        t_n = time_values[n]
        
        # Copy from the next time step to start (we'll update)
        V[n, :, :] = V[n+1, :, :]

        # 5. Perform the explicit update for each interior point i,j
        #    PDE terms:
        #    dV/dt = 1/2 * alpha^2 f^(2beta) * d^2V/df^2
        #          + rho nu alpha f^beta * d^2V/(df dalpha)
        #          + 1/2 * nu^2 alpha^2 * d^2V/dalpha^2
        #          + r f dV/df - r V
        # 
        for j in range(1, N_alpha):       # alpha index
            alpha_j = alpha_values[j]
            for i in range(1, N_f):       # f index
                f_i = f_values[i]

                # -- Second derivative wrt f:
                V_iff = (V[n+1, j, i+1] - 2.0*V[n+1, j, i] + V[n+1, j, i-1]) / (df*df)

                # -- Second derivative wrt alpha:
                V_iaa = (V[n+1, j+1, i] - 2.0*V[n+1, j, i] + V[n+1, j-1, i]) / (dalpha*dalpha)

                # -- Mixed derivative (df dalpha), approximate with central differences:
                #    d^2V/(df dalpha) ~ [V(n+1, j+1, i+1) - V(n+1, j+1, i-1)
                #                        - V(n+1, j-1, i+1) + V(n+1, j-1, i-1)] / (4 * df * dalpha)
                V_if_alpha = (
                    V[n+1, j+1, i+1] - V[n+1, j+1, i-1]
                    - V[n+1, j-1, i+1] + V[n+1, j-1, i-1]
                ) / (4.0 * df * dalpha)

                # -- First derivative wrt f:
                V_if = (V[n+1, j, i+1] - V[n+1, j, i-1]) / (2.0 * df)

                # 6. PDE coefficients
                #    a = 1/2 alpha^2 f^(2beta)
                #    b = rho nu alpha f^beta
                #    c = 1/2 nu^2 alpha^2
                a = 0.5 * alpha_j**2 * (f_i**(2*beta))
                b = rho * nu * alpha_j * (f_i**beta)
                c = 0.5 * (nu**2) * (alpha_j**2)

                # 7. PDE contribution
                dVdt = a * V_iff + b * V_if_alpha + c * V_iaa
                # Advection/drift + discounting terms (with r = 0, these vanish or reduce)
                if r != 0.0:
                    dVdt += r * f_i * V_if - r * V[n+1, j, i]

                # 8. Explicit update
                V[n, j, i] = V[n+1, j, i] + dt * dVdt

        # 9. Re-apply boundary conditions at each time step
        #    f=0 or i=0
        V[n, :, 0] = 0.0  # Call payoff ~ 0 when f=0
        #    f=f_max or i=N_f
        #    For large f, a call behaves ~ f - K (but we keep it simple: copy from next time or set large)
        V[n, :, -1] = (f_max - strike) if (f_max > strike) else 0.0
        #    alpha=0 or j=0
        #    alpha=alpha_max or j=N_alpha
        #    For simplicity, just copy from the previous time slice to avoid instability:
        V[n, 0, :] = V[n, 1, :]
        V[n, -1, :] = V[n, -2, :]

    return f_values, alpha_values, time_values, V


# --- Example usage ---
if __name__ == "__main__":
    f_vals, alpha_vals, t_vals, V_solution = solve_sabr_pde_explicit()

    # V_solution has shape (N_t+1, N_alpha+1, N_f+1).
    # V_solution[0, :, :] is the solution at t=0.
    # We might want to extract e.g. the solution at alpha = some index, f-grid:
    # Below: pick alpha=some midpoint
    alpha_mid_index = len(alpha_vals)//2
    price_slice = V_solution[0, alpha_mid_index, :]


In [97]:
strike_index = pd.Series(f_vals).loc[pd.Series(f_vals)==100].index[0]
alpha_index = pd.Series(alpha_vals).loc[pd.Series(alpha_vals)==0.2].index[0]
print(V_solution[0, alpha_index, strike_index])

8.159381490052555
